In [4]:
!pip install numpy pandas
!pip install datasets
!pip install transformers

In [5]:
from datasets import load_dataset

# Load the IMDb dataset
dataset = load_dataset("imdb")

# Print dataset info
print(dataset)

# Get a sample of the dataset
sample = dataset['train'][0]
print(sample)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})
{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and

In [6]:
import pandas as pd

In [7]:
# Convert the dataset to a pandas DataFrame
df = pd.DataFrame(dataset['train'])

# Check for missing values
print("Missing values:")
print(df.isnull().sum())

Missing values:
text     0
label    0
dtype: int64
